<a href="https://colab.research.google.com/github/MakkiFourati/RecSys-Thesis/blob/main/Retrieval_Ranking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install turicreate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np
import time
import turicreate as tc
from datetime import date, datetime
from sklearn.model_selection import train_test_split

# 1 Candidate Retrieval

## 1-1 Positive Samples

In [ ]:
df_t = pd.read_csv('drive/MyDrive/Thesis/data/transactions_train.csv')

In [ ]:
df_t

,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,0.050831,2
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023,0.030492,2
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004,0.015237,2
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687003,0.016932,2
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687004,0.016932,2
...,...,...,...,...,...
31788319,2020-09-22,fff2282977442e327b45d8c89afde25617d00124d0f999...,929511001,0.059305,2
31788320,2020-09-22,fff2282977442e327b45d8c89afde25617d00124d0f999...,891322004,0.042356,2
31788321,2020-09-22,fff380805474b287b05cb2a7507b9a013482f7dd0bce0e...,918325001,0.043203,1
31788322,2020-09-22,fff4d3a8b1f3b60af93e78c30a7cb4cf75edaf2590d3e5...,833459002,0.006763,1


In [ ]:
df_t.t_dat = pd.to_datetime(df_t.t_dat)
df_t.dtypes

t_dat               datetime64[ns]
customer_id                 object
article_id                   int64
price                      float64
sales_channel_id             int64
dtype: object

In [ ]:
#df_train = df_t[(df_t['t_dat'] >= datetime(2020,8,19)) & (df_t['t_dat'] <= datetime(2020,9,15))]
#df_val = df_t[(df_t['t_dat']>=datetime(2020,9,16))]
df_train = df_t[df_t['t_dat'] >= datetime(2020,8,5)]

In [ ]:
del df_t

In [ ]:
def week_encode(d):
  if d <= datetime(2020,8,11):
    return 1
  elif d <= datetime(2020,8,18):
    return 2
  elif d <= datetime(2020,8,25):
    return 3
  elif d <= datetime(2020,9,1):
    return 4
  elif d <= datetime(2020,9,8):
    return 5
  elif d <= datetime(2020,9,15):
    return 6
  else:
    return 7

In [ ]:
df_train = df_train.drop_duplicates(subset=df_train.columns.to_list())

In [ ]:
df_train['week'] = df_train['t_dat'].apply(week_encode)
df_train.sort_values('t_dat')

,t_dat,customer_id,article_id,price,sales_channel_id,week
29933918,2020-08-05,0011e0bd4c39195ff342c0ca0ac0601ce2b943a8265506...,751471042,0.032746,2,1
29969470,2020-08-05,aa0779c1a1dcd4bae98731cf10c9be8070ff102a998f4e...,749615006,0.020322,2,1
29969471,2020-08-05,aa0779c1a1dcd4bae98731cf10c9be8070ff102a998f4e...,790564001,0.025407,2,1
29969472,2020-08-05,aa0779c1a1dcd4bae98731cf10c9be8070ff102a998f4e...,861558003,0.016932,2,1
29969473,2020-08-05,aa0779c1a1dcd4bae98731cf10c9be8070ff102a998f4e...,918547001,0.008458,2,1
...,...,...,...,...,...,...
31766357,2020-09-22,53da4b44e81286ed175a46d8ffd5a2baf47843089dc03f...,874169001,0.042356,1,7
31766356,2020-09-22,53da4b44e81286ed175a46d8ffd5a2baf47843089dc03f...,793506006,0.025407,1,7
31766355,2020-09-22,53d894ef004472123c51784d6de628ae9efbb823c1295b...,904416001,0.033881,2,7
31766352,2020-09-22,53d5f95331b01525404c3cbb2da6a84e1173dccb979d28...,752814020,0.033881,2,7


In [ ]:
df_train.groupby('week')['t_dat'].count()

week
1    263971
2    242131
3    234712
4    259513
5    242228
6    232308
7    217916
Name: t_dat, dtype: int64

In [ ]:
tmp = df_train[['customer_id','week']].groupby('customer_id',as_index=False).max()

In [ ]:
#df_positives = df_train.merge(tmp, on = ['customer_id', 'week']).loc[:,['customer_id','article_id']]
df_positives = df_train.merge(tmp, on = ['customer_id', 'week'])#.drop(columns=['t_dat','week'])

In [ ]:
df_positives

In [ ]:
df_positives['label'] = 1

In [ ]:
df_train.shape

(1692779, 6)

## 1-2 Negative Samples

### Repurchase

In [ ]:
df_positives = df_positives.merge(df_train,how='right',on=df_train.columns.to_list())
df_positives

,t_dat,customer_id,article_id,price,sales_channel_id,week,label
0,2020-08-05,0011e0bd4c39195ff342c0ca0ac0601ce2b943a8265506...,751471042,0.032746,2,1,NaN
1,2020-08-05,0011e0bd4c39195ff342c0ca0ac0601ce2b943a8265506...,824499006,0.065525,2,1,NaN
2,2020-08-05,0011e0bd4c39195ff342c0ca0ac0601ce2b943a8265506...,913688001,0.032746,2,1,NaN
3,2020-08-05,0013bde09d10db6b0a6a3b0987ac60b643013dfc6f924b...,776237020,0.025407,2,1,1.0
4,2020-08-05,0013bde09d10db6b0a6a3b0987ac60b643013dfc6f924b...,599580055,0.016932,2,1,1.0
...,...,...,...,...,...,...,...
1692774,2020-09-22,fff2282977442e327b45d8c89afde25617d00124d0f999...,929511001,0.059305,2,7,1.0
1692775,2020-09-22,fff2282977442e327b45d8c89afde25617d00124d0f999...,891322004,0.042356,2,7,1.0
1692776,2020-09-22,fff380805474b287b05cb2a7507b9a013482f7dd0bce0e...,918325001,0.043203,1,7,1.0
1692777,2020-09-22,fff4d3a8b1f3b60af93e78c30a7cb4cf75edaf2590d3e5...,833459002,0.006763,1,7,1.0


In [ ]:
df_positives['label'] = df_positives['label'].fillna(0)

In [ ]:
df_positives.groupby('label').count()

,t_dat,customer_id,article_id,price,sales_channel_id,week
label,,,,,,
0.0,604093,604093,604093,604093,604093,604093
1.0,1088686,1088686,1088686,1088686,1088686,1088686


In [ ]:
del tmp

In [ ]:
df_positives['CF'] = 0
df_positives['CF_score'] = 0
df_positives

,t_dat,customer_id,article_id,price,sales_channel_id,week,label,CF,CF_score
0,2020-08-05,0011e0bd4c39195ff342c0ca0ac0601ce2b943a8265506...,751471042,0.032746,2,1,0.0,0,0
1,2020-08-05,0011e0bd4c39195ff342c0ca0ac0601ce2b943a8265506...,824499006,0.065525,2,1,0.0,0,0
2,2020-08-05,0011e0bd4c39195ff342c0ca0ac0601ce2b943a8265506...,913688001,0.032746,2,1,0.0,0,0
3,2020-08-05,0013bde09d10db6b0a6a3b0987ac60b643013dfc6f924b...,776237020,0.025407,2,1,1.0,0,0
4,2020-08-05,0013bde09d10db6b0a6a3b0987ac60b643013dfc6f924b...,599580055,0.016932,2,1,1.0,0,0
...,...,...,...,...,...,...,...,...,...
1692774,2020-09-22,fff2282977442e327b45d8c89afde25617d00124d0f999...,929511001,0.059305,2,7,1.0,0,0
1692775,2020-09-22,fff2282977442e327b45d8c89afde25617d00124d0f999...,891322004,0.042356,2,7,1.0,0,0
1692776,2020-09-22,fff380805474b287b05cb2a7507b9a013482f7dd0bce0e...,918325001,0.043203,1,7,1.0,0,0
1692777,2020-09-22,fff4d3a8b1f3b60af93e78c30a7cb4cf75edaf2590d3e5...,833459002,0.006763,1,7,1.0,0,0


In [ ]:
df_positives['repurchase'] = df_positives['label'].apply(lambda x: 0 if x == 1 else 1)

In [ ]:
df_positives.drop(columns=['t_dat'],inplace=True)
df_positives

,customer_id,article_id,price,sales_channel_id,week,label,CF,CF_score,repurchase
0,0011e0bd4c39195ff342c0ca0ac0601ce2b943a8265506...,751471042,0.032746,2,1,0.0,0,0,1
1,0011e0bd4c39195ff342c0ca0ac0601ce2b943a8265506...,824499006,0.065525,2,1,0.0,0,0,1
2,0011e0bd4c39195ff342c0ca0ac0601ce2b943a8265506...,913688001,0.032746,2,1,0.0,0,0,1
3,0013bde09d10db6b0a6a3b0987ac60b643013dfc6f924b...,776237020,0.025407,2,1,1.0,0,0,0
4,0013bde09d10db6b0a6a3b0987ac60b643013dfc6f924b...,599580055,0.016932,2,1,1.0,0,0,0
...,...,...,...,...,...,...,...,...,...
1692774,fff2282977442e327b45d8c89afde25617d00124d0f999...,929511001,0.059305,2,7,1.0,0,0,0
1692775,fff2282977442e327b45d8c89afde25617d00124d0f999...,891322004,0.042356,2,7,1.0,0,0,0
1692776,fff380805474b287b05cb2a7507b9a013482f7dd0bce0e...,918325001,0.043203,1,7,1.0,0,0,0
1692777,fff4d3a8b1f3b60af93e78c30a7cb4cf75edaf2590d3e5...,833459002,0.006763,1,7,1.0,0,0,0


### 1-2-1 Collaborative Filtering

In [ ]:
users_ids = df_train['customer_id'].unique()
users_dic = {u:j for u,j in zip(users_ids,range(len(users_ids)))}

In [ ]:
items_ids = df_train['article_id'].unique()
items_dic = {i:j for i,j in zip(items_ids ,range(len(items_ids)))}

In [ ]:
df_CF_train = df_train.copy()
df_CF_train['customer_id'] = df_CF_train['customer_id'].map(users_dic)
df_CF_train['article_id'] = df_CF_train['article_id'].map(items_dic)
df_CF_train

,t_dat,customer_id,article_id,price,sales_channel_id,week
29933918,2020-08-05,0,0,0.032746,2,1
29933919,2020-08-05,0,1,0.065525,2,1
29933921,2020-08-05,0,2,0.032746,2,1
29933922,2020-08-05,1,3,0.025407,2,1
29933923,2020-08-05,1,4,0.016932,2,1
...,...,...,...,...,...,...
31788319,2020-09-22,30350,30882,0.059305,2,7
31788320,2020-09-22,30350,1849,0.042356,2,7
31788321,2020-09-22,234034,33320,0.043203,1,7
31788322,2020-09-22,65267,11952,0.006763,1,7


In [ ]:
# create an SFrame out of df
dfS = tc.SFrame(df_CF_train.iloc[:,1:3])

In [ ]:
del df_CF_train

In [ ]:
N_CF_ITEMS = 20

#### Cosine Similarity

In [ ]:
cosine = tc.item_similarity_recommender.create(dfS_train, 
                                              user_id='customer_id', 
                                              item_id='article_id',
                                              similarity_type='cosine')

Preparing data set.

Data has 1149781 observations with 249511 users and 29694 items.

Data prepared in: 1.45085s

Training model from provided data.

Gathering per-item and per-user statistics.

...

| 10.88s                              | 100              | 29694           |

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 12.0005s

In [ ]:
# recommend k products for a list of users (None corresponds to all users) 
recommendation_cos = cosine.recommend(users=None, k=N_CF_ITEMS)

recommendations finished on 1000/249511 queries. users per second: 2552.44

recommendations finished on 2000/249511 queries. users per second: 2579.36

recommendations finished on 3000/249511 queries. users per second: 2525.79

recommendations finished on 4000/249511 queries. users per second: 2541.37

recommendations finished on 5000/249511 queries. users per second: 2568.36

...

recommendations finished on 245000/249511 queries. users per second: 2555.72

recommendations finished on 246000/249511 queries. users per second: 2555.81

recommendations finished on 247000/249511 queries. users per second: 2556.18

recommendations finished on 248000/249511 queries. users per second: 2556.49

recommendations finished on 249000/249511 queries. users per second: 2556.48

In [ ]:
recommendation_cos.print_rows(30)

+-------------+------------+-----------------------+------+
| customer_id | article_id |         score         | rank |
+-------------+------------+-----------------------+------+
|      0      |    4345    |  0.02265251033446368  |  1   |
|      0      |    380     |  0.02029078848221723  |  2   |
|      0      |    5470    |  0.016634660608628216 |  3   |
|      0      |    1558    |  0.016199441517100614 |  4   |
|      0      |    2593    |  0.013634345110724954 |  5   |
|      0      |     47     |  0.013183127431308521 |  6   |
|      0      |    8115    |  0.011291921138763428 |  7   |
|      0      |    980     |  0.010768031372743495 |  8   |
|      0      |    1633    |  0.009394526481628418 |  9   |
|      0      |    4178    |  0.00925880319931928  |  10  |
|      0      |   20144    |  0.008978885762831745 |  11  |
|      0      |   17340    |  0.008935935357037713 |  12  |
|      0      |    2289    |  0.008660803822910084 |  13  |
|      0      |    8116    |  0.00857563

In [ ]:
# evaluating the model
cos_eval = cosine.evaluate_precision_recall(dfS_val,cutoffs=range(1,13))

recommendations finished on 1000/68984 queries. users per second: 1965.29

recommendations finished on 2000/68984 queries. users per second: 1946.11

recommendations finished on 3000/68984 queries. users per second: 1944.04

recommendations finished on 4000/68984 queries. users per second: 1944.08

recommendations finished on 5000/68984 queries. users per second: 1925.54

...

recommendations finished on 64000/68984 queries. users per second: 1931.71

recommendations finished on 65000/68984 queries. users per second: 1929.39

recommendations finished on 66000/68984 queries. users per second: 1929.79

recommendations finished on 67000/68984 queries. users per second: 1928.62

recommendations finished on 68000/68984 queries. users per second: 1929.59

In [ ]:
cos_eval

{'precision_recall_by_user': Columns:
 	customer_id	int
 	cutoff	int
 	precision	float
 	recall	float
 	count	int
 
 Rows: 827808
 
 Data:
 +-------------+--------+-----------+--------+-------+
 | customer_id | cutoff | precision | recall | count |
 +-------------+--------+-----------+--------+-------+
 |      0      |   1    |    0.0    |  0.0   |   9   |
 |      0      |   2    |    0.0    |  0.0   |   9   |
 |      0      |   3    |    0.0    |  0.0   |   9   |
 |      0      |   4    |    0.0    |  0.0   |   9   |
 |      0      |   5    |    0.0    |  0.0   |   9   |
 |      0      |   6    |    0.0    |  0.0   |   9   |
 |      0      |   7    |    0.0    |  0.0   |   9   |
 |      0      |   8    |    0.0    |  0.0   |   9   |
 |      0      |   9    |    0.0    |  0.0   |   9   |
 |      0      |   10   |    0.0    |  0.0   |   9   |
 +-------------+--------+-----------+--------+-------+
 [827808 rows x 5 columns]
 Note: Only the head of the SFrame is printed.
 You can use prin

In [ ]:
cos_eval['precision_recall_overall'][11]

{'cutoff': 12,
 'precision': 0.003494771734508516,
 'recall': 0.015252726430950244}

#### Jaccard similarity

In [ ]:
jaccard = tc.item_similarity_recommender.create(dfS_train, 
                                              user_id='customer_id', 
                                              item_id='article_id',
                                              similarity_type='jaccard')
# recommend k products for a list of users (None corresponds to all users) 
recommendation_jac = jaccard.recommend(users=None, k=N_CF_ITEMS)

Preparing data set.

Data has 2471174 observations with 407764 users and 37100 items.

Data prepared in: 1.23173s

Training model from provided data.

Gathering per-item and per-user statistics.

...

recommendations finished on 403000/407764 queries. users per second: 6397.76

recommendations finished on 404000/407764 queries. users per second: 6397.94

recommendations finished on 405000/407764 queries. users per second: 6398.01

recommendations finished on 406000/407764 queries. users per second: 6397.84

recommendations finished on 407000/407764 queries. users per second: 6397.06

In [ ]:
# print the n first recommendations
recommendation_jac.print_rows(20)

+-------------+------------+----------------------+------+
| customer_id | article_id |        score         | rank |
+-------------+------------+----------------------+------+
|      0      |    1920    | 0.15151512622833252  |  1   |
|      0      |    2250    | 0.01571708917617798  |  2   |
|      0      |   25808    | 0.015126049518585205 |  3   |
|      0      |   26708    | 0.015009403228759766 |  4   |
|      0      |    613     | 0.01360541582107544  |  5   |
|      0      |    845     | 0.012396693229675293 |  6   |
|      0      |   16736    | 0.012254893779754639 |  7   |
|      0      |    6239    | 0.012096762657165527 |  8   |
|      0      |   17289    | 0.011627912521362305 |  9   |
|      0      |   26694    | 0.011142075061798096 |  10  |
|      0      |    1209    | 0.011009156703948975 |  11  |
|      0      |   25684    | 0.010856449604034424 |  12  |
|      0      |   18995    | 0.010416686534881592 |  13  |
|      0      |    934     | 0.010330557823181152 |  14 

In [ ]:
jac_eval = jaccard.evaluate_precision_recall(dfS_val, cutoffs = range(1,13))

recommendations finished on 1000/68984 queries. users per second: 4535.21

recommendations finished on 2000/68984 queries. users per second: 4602.11

recommendations finished on 3000/68984 queries. users per second: 4609.93

recommendations finished on 4000/68984 queries. users per second: 4637.04

recommendations finished on 5000/68984 queries. users per second: 4617.84

...

recommendations finished on 64000/68984 queries. users per second: 4612.64

recommendations finished on 65000/68984 queries. users per second: 4612.08

recommendations finished on 66000/68984 queries. users per second: 4613.05

recommendations finished on 67000/68984 queries. users per second: 4611.71

recommendations finished on 68000/68984 queries. users per second: 4613.64

In [ ]:
jac_eval

{'precision_recall_by_user': Columns:
 	customer_id	int
 	cutoff	int
 	precision	float
 	recall	float
 	count	int
 
 Rows: 827808
 
 Data:
 +-------------+--------+-----------+--------+-------+
 | customer_id | cutoff | precision | recall | count |
 +-------------+--------+-----------+--------+-------+
 |      15     |   1    |    0.0    |  0.0   |   4   |
 |      15     |   2    |    0.0    |  0.0   |   4   |
 |      15     |   3    |    0.0    |  0.0   |   4   |
 |      15     |   4    |    0.0    |  0.0   |   4   |
 |      15     |   5    |    0.0    |  0.0   |   4   |
 |      15     |   6    |    0.0    |  0.0   |   4   |
 |      15     |   7    |    0.0    |  0.0   |   4   |
 |      15     |   8    |    0.0    |  0.0   |   4   |
 |      15     |   9    |    0.0    |  0.0   |   4   |
 |      15     |   10   |    0.0    |  0.0   |   4   |
 +-------------+--------+-----------+--------+-------+
 [827808 rows x 5 columns]
 Note: Only the head of the SFrame is printed.
 You can use prin

In [ ]:
jac_eval['precision_recall_overall'][11]

{'cutoff': 12,
 'precision': 0.0044672194518535786,
 'recall': 0.018802517576061564}

#### Matrix Factorization

In [ ]:
# Due to the poor performance of MF in this project, I chose not to include it here

#### Choosing the best CF model

In [ ]:
#training on the whole data
jaccard = tc.item_similarity_recommender.create(dfS, 
                                              user_id='customer_id', 
                                              item_id='article_id',
                                              similarity_type='jaccard')
# recommend k products for a list of users (None corresponds to all users) 
recommendation_jac = jaccard.recommend(users=None, k=N_CF_ITEMS)

Preparing data set.

Data has 1692779 observations with 343677 users and 33971 items.

Data prepared in: 1.03618s

Training model from provided data.

Gathering per-item and per-user statistics.

...

recommendations finished on 339000/343677 queries. users per second: 7278.33

recommendations finished on 340000/343677 queries. users per second: 7278.52

recommendations finished on 341000/343677 queries. users per second: 7278.78

recommendations finished on 342000/343677 queries. users per second: 7277.95

recommendations finished on 343000/343677 queries. users per second: 7263.78

In [ ]:
recommendation_jac.print_rows(100)

### 1-2-2 Popularity

In [ ]:
# Creating a dataset with the N most purchased items during the last 5 weeks
N_MOST_POPULAR = 100
df_popularity = df_train.groupby('article_id',as_index=False).count().iloc[:,:2]
df_popularity.rename(columns={'t_dat': 'popularity_score'},inplace=True)
df_best_N = df_popularity.sort_values(by=['popularity_score'], 
                                      ascending= False).iloc[:N_MOST_POPULAR]


In [ ]:
df_best_N

,article_id,popularity_score
10638,751471001,3302
32728,918292001,3082
7221,706016001,2794
32513,915526001,2678
10657,751471043,2598
...,...,...
16245,806388001,1065
28631,884319003,1064
11838,762846008,1061
31571,905957001,1053


In [ ]:
df_best_N_CF = df_best_N.merge(df_train[['customer_id']].drop_duplicates(),
                                how='cross')
   

In [ ]:
df_best_N_CF

,article_id,popularity_score,customer_id
0,751471001,3302,0011e0bd4c39195ff342c0ca0ac0601ce2b943a8265506...
1,751471001,3302,0013bde09d10db6b0a6a3b0987ac60b643013dfc6f924b...
2,751471001,3302,002064c9fbf10c4519c8f9aba4a295bb117e8381d6308e...
3,751471001,3302,002c6ef01000772abd545032fd61853b2dc06210299455...
4,751471001,3302,002ccdddcdc2113cfc8840afe62f05a8edce57a66a8522...
...,...,...,...
27494155,456163060,1143,ff5b8a8b26bf93a66290e9bd1b73393ac6a58968a78519...
27494156,456163060,1143,ff6f55a51af284b71dcd264396b299e548f968c1769e71...
27494157,456163060,1143,ff732b98019465f078144bae647d7d9765d76e3431ef92...
27494158,456163060,1143,ffb72741f3bc3d98855703b55d34e05bc7893a5d6a99a3...


In [ ]:
pop_dic = {k:v for k,v in zip(df_popularity['article_id'].tolist(),df_popularity['popularity_score'].tolist())}

# 2 Ranking

## 2-1 Preprocessing

In [ ]:
recommendation_jac

customer_id,article_id,score,rank
0,31008,0.02474457025527954,1
0,29442,0.01845690608024597,2
0,2075,0.015740762154261272,3
0,3815,0.01178338130315145,4
0,1052,0.01123457153638204,5
0,14888,0.009359250466028849,6
0,12375,0.008570382992426554,7
0,6889,0.007480134566624959,8
0,29475,0.00682218869527181,9
0,4810,0.0062056779861450195,10


In [ ]:
df_CF = recommendation_jac.to_dataframe()
df_CF

,customer_id,article_id,score,rank
0,0,31008,0.024745,1
1,0,29442,0.018457,2
2,0,2075,0.015741,3
3,0,3815,0.011783,4
4,0,1052,0.011235,5
...,...,...,...,...
6873535,343676,7794,0.010417,16
6873536,343676,31890,0.010417,17
6873537,343676,7843,0.010204,18
6873538,343676,15724,0.010204,19


In [ ]:
df_CF = df_CF.iloc[:,:3]
df_CF['CF'] = 1
df_CF['label'] = 0
df_CF = df_CF.rename(columns={'score': 'CF_score'})

In [ ]:
inv_users_dic = {v: k for k, v in users_dic.items()}
inv_items_dic = {v: k for k, v in items_dic.items()}

df_CF['customer_id'] = df_CF['customer_id'].map(inv_users_dic)
df_CF['article_id'] = df_CF['article_id'].map(inv_items_dic)

In [ ]:
df_CF

,customer_id,article_id,CF_score,CF,label
0,0011e0bd4c39195ff342c0ca0ac0601ce2b943a8265506...,923758001,0.024745,1,0
1,0011e0bd4c39195ff342c0ca0ac0601ce2b943a8265506...,924243002,0.018457,1,0
2,0011e0bd4c39195ff342c0ca0ac0601ce2b943a8265506...,913688003,0.015741,1,0
3,0011e0bd4c39195ff342c0ca0ac0601ce2b943a8265506...,751471001,0.011783,1,0
4,0011e0bd4c39195ff342c0ca0ac0601ce2b943a8265506...,751471041,0.011235,1,0
...,...,...,...,...,...
6873535,ffc92c3f7b0b302f393c2968b290f6e5c5b5510d1cf1df...,780213001,0.010417,1,0
6873536,ffc92c3f7b0b302f393c2968b290f6e5c5b5510d1cf1df...,807241050,0.010417,1,0
6873537,ffc92c3f7b0b302f393c2968b290f6e5c5b5510d1cf1df...,824208003,0.010204,1,0
6873538,ffc92c3f7b0b302f393c2968b290f6e5c5b5510d1cf1df...,894901001,0.010204,1,0


In [ ]:
df_positives

,customer_id,article_id,price,sales_channel_id,week,label,CF,CF_score,repurchase
0,0011e0bd4c39195ff342c0ca0ac0601ce2b943a8265506...,751471042,0.032746,2,1,0.0,0,0,1
1,0011e0bd4c39195ff342c0ca0ac0601ce2b943a8265506...,824499006,0.065525,2,1,0.0,0,0,1
2,0011e0bd4c39195ff342c0ca0ac0601ce2b943a8265506...,913688001,0.032746,2,1,0.0,0,0,1
3,0013bde09d10db6b0a6a3b0987ac60b643013dfc6f924b...,776237020,0.025407,2,1,1.0,0,0,0
4,0013bde09d10db6b0a6a3b0987ac60b643013dfc6f924b...,599580055,0.016932,2,1,1.0,0,0,0
...,...,...,...,...,...,...,...,...,...
1692774,fff2282977442e327b45d8c89afde25617d00124d0f999...,929511001,0.059305,2,7,1.0,0,0,0
1692775,fff2282977442e327b45d8c89afde25617d00124d0f999...,891322004,0.042356,2,7,1.0,0,0,0
1692776,fff380805474b287b05cb2a7507b9a013482f7dd0bce0e...,918325001,0.043203,1,7,1.0,0,0,0
1692777,fff4d3a8b1f3b60af93e78c30a7cb4cf75edaf2590d3e5...,833459002,0.006763,1,7,1.0,0,0,0


In [ ]:
#concatenating positive samples to CF samples
df = pd.concat([df_positives,df_CF])
df['popularity_score'] = df['article_id'].map(pop_dic)
df

,customer_id,article_id,price,sales_channel_id,week,label,CF,CF_score,repurchase,popularity_score
0,0011e0bd4c39195ff342c0ca0ac0601ce2b943a8265506...,751471042,0.032746,2.0,1.0,0.0,0,0.000000,1.0,673
1,0011e0bd4c39195ff342c0ca0ac0601ce2b943a8265506...,824499006,0.065525,2.0,1.0,0.0,0,0.000000,1.0,200
2,0011e0bd4c39195ff342c0ca0ac0601ce2b943a8265506...,913688001,0.032746,2.0,1.0,0.0,0,0.000000,1.0,400
3,0013bde09d10db6b0a6a3b0987ac60b643013dfc6f924b...,776237020,0.025407,2.0,1.0,1.0,0,0.000000,0.0,320
4,0013bde09d10db6b0a6a3b0987ac60b643013dfc6f924b...,599580055,0.016932,2.0,1.0,1.0,0,0.000000,0.0,259
...,...,...,...,...,...,...,...,...,...,...
6873535,ffc92c3f7b0b302f393c2968b290f6e5c5b5510d1cf1df...,780213001,NaN,NaN,NaN,0.0,1,0.010417,NaN,15
6873536,ffc92c3f7b0b302f393c2968b290f6e5c5b5510d1cf1df...,807241050,NaN,NaN,NaN,0.0,1,0.010417,NaN,14
6873537,ffc92c3f7b0b302f393c2968b290f6e5c5b5510d1cf1df...,824208003,NaN,NaN,NaN,0.0,1,0.010204,NaN,17
6873538,ffc92c3f7b0b302f393c2968b290f6e5c5b5510d1cf1df...,894901001,NaN,NaN,NaN,0.0,1,0.010204,NaN,17


In [ ]:
del df_popularity
del pop_dic

In [ ]:
df = df.merge(df_best_N_CF, how = 'outer', on = df_best_N_CF.columns.to_list())
df

,customer_id,article_id,price,sales_channel_id,week,label,CF,CF_score,repurchase,popularity_score
0,0011e0bd4c39195ff342c0ca0ac0601ce2b943a8265506...,751471042,0.032746,2.0,1.0,0.0,0.0,0.0,1.0,673
1,0011e0bd4c39195ff342c0ca0ac0601ce2b943a8265506...,824499006,0.065525,2.0,1.0,0.0,0.0,0.0,1.0,200
2,0011e0bd4c39195ff342c0ca0ac0601ce2b943a8265506...,913688001,0.032746,2.0,1.0,0.0,0.0,0.0,1.0,400
3,0013bde09d10db6b0a6a3b0987ac60b643013dfc6f924b...,776237020,0.025407,2.0,1.0,1.0,0.0,0.0,0.0,320
4,0013bde09d10db6b0a6a3b0987ac60b643013dfc6f924b...,599580055,0.016932,2.0,1.0,1.0,0.0,0.0,0.0,259
...,...,...,...,...,...,...,...,...,...,...
35533975,ff5b8a8b26bf93a66290e9bd1b73393ac6a58968a78519...,456163060,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1143
35533976,ff6f55a51af284b71dcd264396b299e548f968c1769e71...,456163060,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1143
35533977,ff732b98019465f078144bae647d7d9765d76e3431ef92...,456163060,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1143
35533978,ffb72741f3bc3d98855703b55d34e05bc7893a5d6a99a3...,456163060,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1143


In [ ]:
df=df.fillna({'price':0,'sales_channel_id':0,'week':0
              ,'repurchase':0,'CF_score':0, 'CF':0,'label':0})
df

,customer_id,article_id,price,sales_channel_id,week,label,CF,CF_score,repurchase,popularity_score
0,0011e0bd4c39195ff342c0ca0ac0601ce2b943a8265506...,751471042,0.032746,2.0,1.0,0.0,0.0,0.0,1.0,673
1,0011e0bd4c39195ff342c0ca0ac0601ce2b943a8265506...,824499006,0.065525,2.0,1.0,0.0,0.0,0.0,1.0,200
2,0011e0bd4c39195ff342c0ca0ac0601ce2b943a8265506...,913688001,0.032746,2.0,1.0,0.0,0.0,0.0,1.0,400
3,0013bde09d10db6b0a6a3b0987ac60b643013dfc6f924b...,776237020,0.025407,2.0,1.0,1.0,0.0,0.0,0.0,320
4,0013bde09d10db6b0a6a3b0987ac60b643013dfc6f924b...,599580055,0.016932,2.0,1.0,1.0,0.0,0.0,0.0,259
...,...,...,...,...,...,...,...,...,...,...
35533975,ff5b8a8b26bf93a66290e9bd1b73393ac6a58968a78519...,456163060,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,1143
35533976,ff6f55a51af284b71dcd264396b299e548f968c1769e71...,456163060,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,1143
35533977,ff732b98019465f078144bae647d7d9765d76e3431ef92...,456163060,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,1143
35533978,ffb72741f3bc3d98855703b55d34e05bc7893a5d6a99a3...,456163060,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,1143


In [ ]:
df.isna().any()

customer_id         False
article_id          False
price               False
sales_channel_id    False
week                False
label               False
CF                  False
CF_score            False
repurchase          False
popularity_score    False
dtype: bool

In [ ]:
df.to_csv('ranking.csv')

In [ ]:
df_cust = pd.read_csv('drive/MyDrive/Thesis/data/customers_cleaned_full.csv')
df_art = pd.read_csv('drive/MyDrive/Thesis/data/articles_num.csv')

In [ ]:
df = df.merge(df_cust, on = 'customer_id')

In [ ]:
del df_cust

In [ ]:
df = df.merge(df_art[['article_id', 'product_type_no', 
                      'perceived_colour_master_id','index_group_no']],
              on = 'article_id')

In [ ]:
df = df.sort_values('customer_id')

In [ ]:
df

,customer_id,article_id,price,sales_channel_id,week,label,CF,CF_score,repurchase,popularity_score,age,postal_code,product_type_no,perceived_colour_master_id,index_group_no
4124087,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,915526001,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,2678,49.0,112978,252,9,1
26810942,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,706016002,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,1151,49.0,112978,272,2,2
33313074,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,858856005,0.000000,0.0,0.0,0.0,1.0,0.097682,0.0,174,49.0,112978,275,19,1
25436086,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,572998009,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,1164,49.0,112978,272,2,2
16155695,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,579541001,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,1389,49.0,112978,245,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10335674,ffffcf35913a0bee60e8741cb2b4e78b8a98ee5ff2e6a1...,758034001,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,1695,21.0,95707,286,5,1
34498276,ffffcf35913a0bee60e8741cb2b4e78b8a98ee5ff2e6a1...,689365050,0.010017,2.0,5.0,0.0,0.0,0.000000,1.0,96,21.0,95707,259,4,2
15148126,ffffcf35913a0bee60e8741cb2b4e78b8a98ee5ff2e6a1...,806388002,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,1469,21.0,95707,255,9,2
28251894,ffffcf35913a0bee60e8741cb2b4e78b8a98ee5ff2e6a1...,889460003,0.000000,0.0,0.0,0.0,1.0,0.010469,0.0,858,21.0,95707,258,1,1


In [ ]:
df.reset_index(drop=True,inplace=True)

In [ ]:
df.to_csv('ranking_train.csv',index=False)

## 2-2 GPU Accelerator

In [ ]:
!rm -r /opt/conda/lib/python3.6/site-packages/lightgbm
!git clone --recursive https://github.com/Microsoft/LightGBM

rm: cannot remove '/opt/conda/lib/python3.6/site-packages/lightgbm': No such file or directory
fatal: destination path 'LightGBM' already exists and is not an empty directory.


In [ ]:
!apt-get install -y libboost-all-dev
#Note: If you running this script directly from terminal make sure to remove "!"

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libboost-all-dev is already the newest version (1.65.1.0ubuntu1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 20 not upgraded.


In [ ]:
!apt-get update --fix-missing
#Note: If you running this script directly from terminal make sure to remove "!"

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [913 kB]
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:14 http://ppa

In [ ]:
#/usr/local/cuda/lib64/libOpenCL.so.1

In [ ]:
%%bash
cd LightGBM
#rm -r build
mkdir build
cd build
cmake -DUSE_GPU=1 -DOpenCL_LIBRARY=/usr/local/cuda/lib64/libOpenCL.so.1 -DOpenCL_INCLUDE_DIR=/usr/local/cuda/include/ ..
make -j$(nproc)

-- OpenCL include directory: /usr/local/cuda/include
-- Using _mm_prefetch
-- Using _mm_malloc
-- Configuring done
-- Generating done
-- Build files have been written to: /content/LightGBM/build
Consolidate compiler generated dependencies of target lightgbm_capi_objs
[  1%] Built target lightgbm_capi_objs
Consolidate compiler generated dependencies of target lightgbm_objs
[ 92%] Built target lightgbm_objs
Consolidate compiler generated dependencies of target lightgbm
[ 94%] Built target _lightgbm
[100%] Built target lightgbm


mkdir: cannot create directory ‘build’: File exists


## 2-3 Model

In [ ]:
import pandas as pd
import numpy as np
import lightgbm
import time
from datetime import date, datetime
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv('ranking_train.csv')
df

,customer_id,article_id,price,sales_channel_id,week,label,CF,CF_score,repurchase,popularity_score,age,postal_code,product_type_no,perceived_colour_master_id,index_group_no
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,915526001,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,2678,49.0,112978,252,9,1
1,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,706016002,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,1151,49.0,112978,272,2,2
2,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,858856005,0.000000,0.0,0.0,0.0,1.0,0.097682,0.0,174,49.0,112978,275,19,1
3,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,572998009,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,1164,49.0,112978,272,2,2
4,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,579541001,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,1389,49.0,112978,245,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35533975,ffffcf35913a0bee60e8741cb2b4e78b8a98ee5ff2e6a1...,758034001,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,1695,21.0,95707,286,5,1
35533976,ffffcf35913a0bee60e8741cb2b4e78b8a98ee5ff2e6a1...,689365050,0.010017,2.0,5.0,0.0,0.0,0.000000,1.0,96,21.0,95707,259,4,2
35533977,ffffcf35913a0bee60e8741cb2b4e78b8a98ee5ff2e6a1...,806388002,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,1469,21.0,95707,255,9,2
35533978,ffffcf35913a0bee60e8741cb2b4e78b8a98ee5ff2e6a1...,889460003,0.000000,0.0,0.0,0.0,1.0,0.010469,0.0,858,21.0,95707,258,1,1


In [ ]:
qids_train = df.groupby('customer_id')['customer_id'].count().to_numpy()
X_train = df.drop(['customer_id', 'label'], axis=1)
y_train = df['label']

#qids_validation = df_val.groupby('customer_id')['customer_id'].count().to_numpy()
#X_validation = df_val.drop(['customer_id', 'label'], axis=1)
#y_validation = df_val['label']

In [ ]:
model = lightgbm.LGBMRanker(
    objective='lambdarank',
    learning_rate=0.001,
    n_estimators=300,
    num_leaves=40,
    n_jobs=-1,
)

In [ ]:
model.fit(
    X=X_train,
    y=y_train,
    group=qids_train,
    #eval_set=[(X_validation, y_validation)],
    #eval_group=[qids_validation],
    #eval_at=12,
    verbose=10,
)

LGBMRanker(learning_rate=0.001, n_estimators=300, num_leaves=40,
           objective='lambdarank')

In [ ]:
import pickle

with open("rank-model.pkl", "wb") as f:
    pickle.dump(model, f)

## Prediction

In [ ]:
import pickle
with open("rank-model.pkl", "rb") as f:
    model = pickle.load(f)

In [ ]:
pred_train = model.predict(X_train)
pred_train

array([-0.1326638, -0.1326638, -0.1326638, ..., -0.1326638, -0.1326638,
       -0.1326638])

In [ ]:
pred_train.shape

(35533980,)

In [ ]:
df_pred_train = df.iloc[:,:2]
df_pred_train['pred'] = pred_train
df_pred_train

,customer_id,article_id,pred
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,915526001,-0.132664
1,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,706016002,-0.132664
2,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,858856005,-0.132664
3,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,572998009,-0.132664
4,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,579541001,-0.132664
...,...,...,...
35533975,ffffcf35913a0bee60e8741cb2b4e78b8a98ee5ff2e6a1...,758034001,-0.132664
35533976,ffffcf35913a0bee60e8741cb2b4e78b8a98ee5ff2e6a1...,689365050,-0.132664
35533977,ffffcf35913a0bee60e8741cb2b4e78b8a98ee5ff2e6a1...,806388002,-0.132664
35533978,ffffcf35913a0bee60e8741cb2b4e78b8a98ee5ff2e6a1...,889460003,-0.132664


In [ ]:
df_pred_train.reset_index(drop=True,inplace=True)

In [ ]:
df_pred_train.sort_values(['customer_id','pred'],ascending=False,inplace= True)

In [ ]:
df_pred_train

,customer_id,article_id,pred
35533930,ffffcf35913a0bee60e8741cb2b4e78b8a98ee5ff2e6a1...,762846027,0.132664
35533887,ffffcf35913a0bee60e8741cb2b4e78b8a98ee5ff2e6a1...,896152001,-0.132664
35533897,ffffcf35913a0bee60e8741cb2b4e78b8a98ee5ff2e6a1...,896152003,-0.132664
35533968,ffffcf35913a0bee60e8741cb2b4e78b8a98ee5ff2e6a1...,896169002,-0.132664
35533912,ffffcf35913a0bee60e8741cb2b4e78b8a98ee5ff2e6a1...,896152002,-0.132664
...,...,...,...
25,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,717490008,-0.132664
69,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,723469001,-0.132664
71,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,916468003,-0.132664
45,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,779781015,-0.132664


In [ ]:
df_pred_train = df_pred_train.groupby('customer_id').head(12)
df_pred_train = df_pred_train.drop(columns=['pred'])
df_pred_train

,customer_id,article_id
35533930,ffffcf35913a0bee60e8741cb2b4e78b8a98ee5ff2e6a1...,762846027
35533887,ffffcf35913a0bee60e8741cb2b4e78b8a98ee5ff2e6a1...,896152001
35533897,ffffcf35913a0bee60e8741cb2b4e78b8a98ee5ff2e6a1...,896152003
35533968,ffffcf35913a0bee60e8741cb2b4e78b8a98ee5ff2e6a1...,896169002
35533912,ffffcf35913a0bee60e8741cb2b4e78b8a98ee5ff2e6a1...,896152002
...,...,...
54,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,884319001
86,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,372860001
58,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,865929003
27,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,863583001


In [ ]:
df_cust = pd.read_csv('drive/MyDrive/Thesis/data/customers_cleaned_full.csv')
df_art = pd.read_csv('drive/MyDrive/Thesis/data/articles_num.csv')

In [ ]:
customers = df_cust['customer_id'].to_list()
customers_pred = df_pred_train['customer_id'].unique()
customers_rmn = list(set(customers) - set(customers_pred))
len(customers_rmn)

1028303

In [ ]:
df_rmn = pd.DataFrame({'customer_id' : customers_rmn})
df_rmn

,customer_id
0,573404df7566bb81d427a28af393dc0fea07f9c64dbd33...
1,c7d6bf885067f3f0c0567f9773c63a07f578fe0486852b...
2,92dd75ce548b4380462f6f5915752be300383fc54add40...
3,c44a2d11c300b457e2900ade701e536fdcaf32f1d78967...
4,87033d3e5137c6d9eff8696e9f51bc7032231f2bc1139d...
...,...
1028298,91ddf428e254cdf11d4962e813f54bb56e0006e03c1406...
1028299,c8220ebe32e6c3808b149ef6afa577ef29d5b32dec6204...
1028300,d6f791bbddaf1d3d986ad31fb76534cbac65196823f510...
1028301,accda919dc33fc06545e2678b40b5110f0745ce533af8d...


In [ ]:
df_rmn['price']=0
df_rmn['sales_channel_id']=0
df_rmn['week']=0
df_rmn['repurchase']=0
df_rmn['CF_score']=0
df_rmn['CF']=0
df_rmn

,customer_id,price,sales_channel_id,week,repurchase,CF_score,CF
0,573404df7566bb81d427a28af393dc0fea07f9c64dbd33...,0,0,0,0,0,0
1,c7d6bf885067f3f0c0567f9773c63a07f578fe0486852b...,0,0,0,0,0,0
2,92dd75ce548b4380462f6f5915752be300383fc54add40...,0,0,0,0,0,0
3,c44a2d11c300b457e2900ade701e536fdcaf32f1d78967...,0,0,0,0,0,0
4,87033d3e5137c6d9eff8696e9f51bc7032231f2bc1139d...,0,0,0,0,0,0
...,...,...,...,...,...,...,...
1028298,91ddf428e254cdf11d4962e813f54bb56e0006e03c1406...,0,0,0,0,0,0
1028299,c8220ebe32e6c3808b149ef6afa577ef29d5b32dec6204...,0,0,0,0,0,0
1028300,d6f791bbddaf1d3d986ad31fb76534cbac65196823f510...,0,0,0,0,0,0
1028301,accda919dc33fc06545e2678b40b5110f0745ce533af8d...,0,0,0,0,0,0


In [ ]:
# get the features names in the same order as for the training set to create 
# prediction set. LightGBM doesn't take into account the names so features must
#be in the same order
features = df.columns.tolist()
features.remove('label')
features

['customer_id',
 'article_id',
 'price',
 'sales_channel_id',
 'week',
 'CF',
 'CF_score',
 'repurchase',
 'popularity_score',
 'age',
 'postal_code',
 'product_type_no',
 'perceived_colour_master_id',
 'index_group_no']

In [ ]:
#We divide our data to be passed to the model into N batches so that the data 
# can fit into memory
N_MOST_POPULAR = 100
nb_batches = 8
nb_cust_rmn = df_rmn.shape[0]
batch_size = int((nb_cust_rmn * N_MOST_POPULAR) / nb_batches)
cust_per_batch = int(nb_cust_rmn / nb_batches) 

In [ ]:
cust_per_batch

128537

In [ ]:
df_rmn_copy = df_rmn.copy()

In [ ]:
l = []

for i in range(nb_batches):
  if i < nb_batches-1:
    df_rmn_i = df_rmn_copy.iloc[:cust_per_batch,:]
    df_rmn_copy = df_rmn_copy[cust_per_batch:]
  else:
    df_rmn_i = df_rmn_copy

  df_rmn_i = df_rmn_i.merge(df_best_N, how='cross')
  df_rmn_i.reset_index(drop=True, inplace= True)


  df_rmn_i = df_rmn_i.merge(df_art[['article_id', 'product_type_no', 
                      'perceived_colour_master_id', 'section_no','index_group_no']],
              on = 'article_id')
  

  df_rmn_i = df_rmn_i.merge(df_cust, on = 'customer_id')
  df_rmn_i = df_rmn_i[features] 

  y = model.predict(df_rmn_i.iloc[:,1:])
  
  df_rmn_i = df_rmn_i.iloc[:,:2]
  df_rmn_i['pred'] = y

  df_rmn_i.sort_values(['customer_id','pred'],ascending=False,inplace= True)
  df_rmn_i = df_rmn_i.groupby('customer_id').head(12)

  l.append(df_rmn_i.drop(columns=['pred']))

  

7


In [ ]:
l.append(df_pred_train)

# Submission

In [ ]:
submission = pd.concat(l,axis=0)
submission

,customer_id,article_id
9977220,ffffaff3905b803d1c7e153a1378a5151e1f34f236ba54...,896169002
9977226,ffffaff3905b803d1c7e153a1378a5151e1f34f236ba54...,896152001
9977240,ffffaff3905b803d1c7e153a1378a5151e1f34f236ba54...,896152003
9977206,ffffaff3905b803d1c7e153a1378a5151e1f34f236ba54...,896152002
9977246,ffffaff3905b803d1c7e153a1378a5151e1f34f236ba54...,874754002
...,...,...
54,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,884319001
86,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,372860001
58,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,865929003
27,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,863583001


In [ ]:
submission['article_id'] = submission['article_id'].apply(lambda x: '0'+str(x))

In [ ]:
submission = submission.groupby(by = 'customer_id', as_index = False).agg({'article_id': ' '.join})
submission

,customer_id,article_id
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0568601043 0896169002 0896152003 0896152001 08...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0896169002 0896152001 0896152003 0896152002 08...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0794321007 0896169002 0896152003 0896152001 08...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,0896169002 0896152001 0896152003 0896152002 08...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,0730683050 0791587015 0927530004 0896152002 08...
...,...,...
1371975,ffffbbf78b6eaac697a8a5dfbfd2bfa8113ee5b403e474...,0804992033 0740922009 0791587007 0720125039 07...
1371976,ffffcd5046a6143d29a04fb8c424ce494a76e5cdf4fab5...,0762846031 0896152001 0896169002 0896152003 08...
1371977,ffffcf35913a0bee60e8741cb2b4e78b8a98ee5ff2e6a1...,0762846027 0896152001 0896152003 0896169002 08...
1371978,ffffd7744cebcf3aca44ae7049d2a94b87074c3d4ffe38...,0896169002 0896152001 0896152003 0896152002 08...


In [ ]:
submission.dtypes

customer_id    object
article_id     object
dtype: object

In [ ]:
len(submission['customer_id'].unique())

1371980

In [ ]:
submission = submission.rename(columns={'article_id':'prediction'})
submission

,customer_id,prediction
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0568601043 0896169002 0896152003 0896152001 08...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0896169002 0896152001 0896152003 0896152002 08...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0794321007 0896169002 0896152003 0896152001 08...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,0896169002 0896152001 0896152003 0896152002 08...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,0730683050 0791587015 0927530004 0896152002 08...
...,...,...
1371975,ffffbbf78b6eaac697a8a5dfbfd2bfa8113ee5b403e474...,0804992033 0740922009 0791587007 0720125039 07...
1371976,ffffcd5046a6143d29a04fb8c424ce494a76e5cdf4fab5...,0762846031 0896152001 0896169002 0896152003 08...
1371977,ffffcf35913a0bee60e8741cb2b4e78b8a98ee5ff2e6a1...,0762846027 0896152001 0896152003 0896169002 08...
1371978,ffffd7744cebcf3aca44ae7049d2a94b87074c3d4ffe38...,0896169002 0896152001 0896152003 0896152002 08...


In [ ]:
submission.to_csv('my_submission.csv',index=False)

In [ ]:
submission['length'] = submission['article_id'].apply(lambda s: len(s))

In [ ]:
submission

,customer_id,article_id,length
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0568601043 0751471001 0751471043 0896152002 09...,131
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0751471001 0706016001 0915526001 0751471043 09...,131
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0794321007 0915526001 0916468003 0909370001 09...,131
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,0751471001 0706016001 0915526001 0751471043 09...,131
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,0751471001 0706016001 0915526001 0751471043 09...,131
...,...,...,...
1371975,ffffbbf78b6eaac697a8a5dfbfd2bfa8113ee5b403e474...,0713997002 0740922009 0791587007 0557599022 07...,131
1371976,ffffcd5046a6143d29a04fb8c424ce494a76e5cdf4fab5...,0751471001 0706016001 0915526001 0751471043 09...,131
1371977,ffffcf35913a0bee60e8741cb2b4e78b8a98ee5ff2e6a1...,0762846027 0915526001 0916468003 0915526002 09...,131
1371978,ffffd7744cebcf3aca44ae7049d2a94b87074c3d4ffe38...,0751471001 0706016001 0915526001 0751471043 09...,131
